**Next Steps**
- Remove softmax
- Combine 3 models by directly summing their predictions with linal norm

- choose between RF and XGB
- choose between M1 variants
- Hierarchical model With custom proba for class 2
- Improve Sampling Strategy


- Add SGD+LR Scheduler
- **Fix OLR**
- Use trained embedding from one simple model to another?

- optimal thresholding


**Aces**

- dropout and regularization
- OLR+SGD+momentum or LR Scheduling
- Architecture and Multiple Receptive Field
    - Go Wide
    - Wider Embedding as well
- HyperParams from Params Section
- Try SGD with LR scheduling
- Try with and without removing hypen `-`
- Optimal thresholding for F1
- epochs based on val acc




**Features**
- Average Rating of Drug
- don't do stop words
- Use Drug Column through embedding layer as well to get Drug Vector.
- Run Fasttext/Glove on this corpus


In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  
 
# Do other imports now...
import keras

import tensorflow as tf
tf.test.is_gpu_available()

from keras import backend as K
K.tensorflow_backend._get_available_gpus()



Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [104]:
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
        del model
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto(log_device_placement=True, allow_soft_placement=True,)
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))
    

In [105]:
reset_keras()

119


# Imports



In [154]:
from keras import backend as K
from keras.layers import Reshape, Multiply
import time
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import KFold
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report
import numpy as np
from nltk import word_tokenize
%matplotlib inline
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, DepthwiseConv2D, Conv2D, SeparableConv2D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, concatenate, LeakyReLU
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Flatten, Dense, Dropout, SpatialDropout1D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l2
%config InlineBackend.figure_format='retina'
from keras_contrib.callbacks import CyclicLR
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from data_science_utils.vision.keras import *
from time import time
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno as msno
import re
from joblib import Parallel, delayed
from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils
from data_science_utils import misc as misc
from data_science_utils import preprocessing as pp_utils
from data_science_utils import nlp as nlp_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from data_science_utils.dataframe import get_specific_cols
from random import sample

import more_itertools
from more_itertools import flatten
import ast
from sklearn.preprocessing import LabelEncoder

import gc
from sklearn.decomposition import PCA

import sys
import os
sys.path.append(os.getcwd())
from importlib import reload


import warnings
warnings.filterwarnings('ignore')

reload(model_utils)
from data_science_utils.models import mean_absolute_percentage_error
from data_science_utils.models import median_absolute_percentage_error
from sklearn.metrics import mean_squared_error
import seaborn as sns
from more_itertools import flatten
import dill
from collections import Counter
import operator
from gensim.models import FastText
import itertools
from gensim.test.utils import datapath
from gensim.utils import tokenize
from gensim.models.fasttext import load_facebook_model

import lib
reload(lib)
from lib import *

import params
reload(params)
from params import *

import data_gen
reload(data_gen)
from data_gen import *

from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.ensemble import RandomForestClassifier

from keras.regularizers import l2
from keras.regularizers import L1L2



<module 'data_science_utils.models' from '/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/data_science_utils/models/__init__.py'>

<module 'lib' from '/home/ec2-user/SageMaker/lib.py'>

<module 'params' from '/home/ec2-user/SageMaker/params.py'>

<module 'data_gen' from '/home/ec2-user/SageMaker/data_gen.py'>

In [4]:
glove = api.load("glove-twitter-25")

In [ ]:

fname = os.getcwd()+"/fasttext.model"
from gensim.models import FastText
fb_model = FastText.load(fname)

[==================================================] 100.0% 104.8/104.8MB downloaded


In [157]:
df = read_csv("train_padded.csv")
df_test = read_csv("test_padded.csv")


df.shape
df_test.shape

(5279, 16)

(2924, 14)

In [158]:
# How many words in test set are not in train set

train_words = set(list(more_itertools.flatten(df.full_txt.values)))
test_words = set(list(more_itertools.flatten(df_test.full_txt.values)))
all_words = set(list(more_itertools.flatten(df.full_txt.values))+list(more_itertools.flatten(df_test.full_txt.values)))
len(train_words)
len(test_words)
test_exlusive_words = test_words-train_words

print("Test Set Exclusive Distinct words = ",len(test_exlusive_words))

print("Total vocab size = ",len(all_words))


25548

18769

Test Set Exclusive Distinct words =  4267
Total vocab size =  29815


# PreProcess for M1 and M5

## Encoding


In [159]:
le_train,le_transform, le = get_text_le(vocab_size=vocab_size, min_count=min_count)
_ = le_train(list(df.full_txt.values)+list(df_test.full_txt.values))

df["full_txt_encoded"] = le_transform(df.full_txt.values)
df_test["full_txt_encoded"] = le_transform(df_test.full_txt.values)

df["context_txt_encoded"] = le_transform(df.context_txt.values)
df_test["context_txt_encoded"] = le_transform(df_test.context_txt.values)

def do_padding(df):
    df['full_txt_encoded'] = list(pad_text_sequences(df['full_txt_encoded'].values,empty=0, maxlen=full_txt_maxlen,jobs=jobs))
    df['context_txt_encoded'] = list(pad_text_sequences(df['context_txt_encoded'].values,empty=0, maxlen=context_txt_maxlen,jobs=jobs))
    return df

df = do_padding(df)
df_test = do_padding(df_test)


 16%|█▋        | 869/5279 [00:00<00:00, 8687.01it/s]

Total Words before Min frequency filtering 29815
Total Words after Min frequency filtering 11859


100%|██████████| 2924/2924 [00:00<00:00, 31016.20it/s]


## Glove Matrix

In [160]:
embedding_matrix = np.random.rand(vocab_size, embedding_dims)*0.05

In [161]:
not_printed=True
for w in le["wd"].keys():
    try:
        embedding_matrix[le["wd"][w]] = glove.wv[w][:embedding_dims]
        if not_printed:
            not_printed=False
            print(w,le["wd"][w])
            print(embedding_matrix[le["wd"][w]])
    except:
        pass


patient 8
[-1.3836      0.71107    -1.83200002  0.65379     0.74203002 -0.1154
  0.47209999 -0.58218998  0.034721    0.17285    -0.51932001  0.26039001
 -2.91829991  0.67479998  0.38541001  0.23262     0.39111     0.49118999
 -0.043787   -1.05869997  0.71371001 -0.11127    -1.02769995  0.13327
 -0.60834998]


# Modelling Functions

In [126]:
def prediction_caliberation_wrapper(arr,weights=[1,1,1]):
    arr = np.copy(arr)
    for i,v in enumerate(weights):
        arr[:,i] = arr[:,i]*v
    return arr


In [21]:
def dense_block(x,layer_width,dropout=0):
    x1 = conv_layer(x,n_kernels=16*layer_width,kernel_size=5,padding='same')
    x2 = conv_layer(x1,n_kernels=16*layer_width,kernel_size=3,padding='same')
    x = concatenate([x1,x2])
    x3 = conv_layer(x,n_kernels=8*layer_width,kernel_size=3,padding='same')
    
    x = concatenate([x1,x2,x3])
    x = transition_layer(x, n_kernels=16*layer_width, dropout=0)
    return x
    

In [22]:
def conv_pipe(x,layer_width,dropout=0):
    
    x1 = dense_block(x,layer_width)
    xg0 = GlobalAveragePooling1D()(x1)
    x2 = dense_block(x1,layer_width)
    xg1 = GlobalAveragePooling1D()(x2)
    
    # x3 = MaxPooling1D()(x2)
    x4 = dense_block(x2,layer_width)
    xg2 = GlobalAveragePooling1D()(x4)
    return concatenate([xg0,xg1, xg2])



In [23]:
def resample_input_for_training(x):
    xs1 = x[x['sentiment']==0]
    xs2 = x[x['sentiment']==1]
    x = pd.concat([x,xs1.sample(frac=1),xs1.sample(frac=1),xs1.sample(frac=1),xs1.sample(frac=1),xs1.sample(frac=1),xs1.sample(frac=1)])
    x = pd.concat([x,xs2.sample(frac=1),xs2.sample(frac=1),xs2.sample(frac=1),xs2.sample(frac=1)])
    x = x.sample(frac=1)
    
    return x


In [122]:
def train_generator_model_with_validation(model,train,val=None,extract_values=None,
                                          epochs=None, batch_size=None,
                                          policy=None,lr=0.01,max_lr_olr=0.005, 
                                          lr_shed_fn=lambda e,lr:0.95*lr):
    # policy olr+sgd | sgd+lrs | adam
    train_copy = train.copy()
    train = resample_input_for_training(train)
    train_iterator = MakeIter(train,extract_values,batch_size)
    print("Before Resampling, Size = ",train_copy.shape,"\n",train_copy.sentiment.value_counts())
    print("After Resampling = ",train.shape,"\n",train.sentiment.value_counts())
    
    
    # checkpoint = ModelCheckpoint("model.hdf5", monitor='val_acc', verbose=0, save_best_only=True, mode='max')
    
    assert epochs is not None
    assert policy is not None
    assert batch_size is not None
    
    if policy == "olr":
        lr_manager = OneCycleLR(samples=train.shape[0], epochs=epochs, batch_size=batch_size,
                            steps=len(train_iterator), max_lr=max_lr_olr,
                            end_percentage=0.2, scale_percentage=None,
                            maximum_momentum=None, minimum_momentum=None)
        callbacks=[lr_manager]
    else:
        callbacks = [LearningRateScheduler(lr_shed_fn)] if lr_shed_fn is not None else []
    
    if policy in ["olr","sgd"]:
        optimizer = SGD(lr=lr, decay=0, momentum=0.9, nesterov=True)
    else:
        optimizer = Adam(lr=lr)
    
    model.compile(loss=["categorical_crossentropy"],
                  optimizer=optimizer,
                  metrics=["acc"],)
    # print(model.summary())
    
    if val is None:
        train_history = model.fit_generator(train_iterator,
                        steps_per_epoch=len(train_iterator), 
                        epochs=epochs, verbose=1,
                        callbacks=callbacks,)
    else:
        validation_iterator = MakeIter(val,extract_values,batch_size)
        metrics = DataGenMetrics(train = MakeIter(train_copy,extract_values,batch_size),
                             val = MakeIter(val,extract_values,batch_size))
        callbacks.append(metrics)
        train_history = model.fit_generator(train_iterator,
                        steps_per_epoch=len(train_iterator), 
                        validation_data = validation_iterator, 
                        validation_steps = len(validation_iterator),
                        epochs=epochs, verbose=1,
                        callbacks=callbacks,)
    

    
    

In [26]:
def extract_test_wrapper(X_Y):
    return X_Y[0]
    
def extract_values(x):
    x1 = np.stack(x["full_txt_encoded"].values, axis=0)
    x2 = np.stack(x["full_txt_mask"].values, axis=0)
    x3 = np.stack(x["full_txt_mask_gaussian"].values, axis=0)
    x4 = np.stack(x["context_txt_encoded"].values, axis=0)
    x5 = x[["occurences_count","review_length","review_length_by_occurences_count"]].values
    train_phase = False
    if 'ohe_labels' in x.columns:
        train_phase = True
        y = np.stack(x['ohe_labels'].values,axis=0)
    
    if train_phase:
        x1,x2,x3 = batch_cutout( x1,x2,x3,p=cutout_proba, min_words=min_cutout, max_words=max_cutout,)
        x4 = batch_cutout(x4, p=cutout_proba, min_words=min_cutout, max_words=max_cutout,)[0]
    if train_phase:
        return [x1,x2,x3,x4,x5],y
    else:
        return [x1,x2,x3,x4,x5]


In [99]:
def looped_random_validation(df,model_fn, training_fn,extract_values,
                             epochs=None, batch_size=None,
                            policy=None,lr=0.01,max_lr_olr=0.005, lr_shed_fn=lambda e,lr:0.95*lr):
    
    
    assert policy is not None
    assert epochs is not None
    assert batch_size is not None
    model = model_fn()

    np.random.seed(8*17 + 3*21+np.random.randint(8754))
    df_train, df_val = train_test_split(df,test_size=0.3)
    training_fn(model,df_train,df_val,extract_values,
               epochs=epochs, batch_size=batch_size, policy=policy,lr=lr,max_lr_olr=max_lr_olr, lr_shed_fn=lr_shed_fn)

    y_train_preds = model.predict(extract_test_wrapper(extract_values(df_train)))
    y_train_preds = np.argmax(y_train_preds, axis=1)
    _ = show_results(df_train['sentiment'], y_train_preds)

    
    y_val_preds = model.predict(extract_test_wrapper(extract_values(df_val)))
    y_val_preds = np.argmax(y_val_preds, axis=1)
    acc,f1_test = show_results(df_val['sentiment'], y_val_preds)
    
    print(classification_report(df_val['sentiment'], y_val_preds))
        
    
    
        
      
        

# M1

In [162]:
def build_model():
    full_txt_input = Input(shape=(full_txt_maxlen,), dtype='int32')
    full_txt_mask = Input(shape=(full_txt_maxlen,), dtype='float32')
    full_txt_gaussian = Input(shape=(full_txt_maxlen,), dtype='float32')
    context_txt_input = Input(shape=(context_txt_maxlen,), dtype='int32')
    
    numeric_inputs = Input(shape=(3,),name="numeric_input", dtype='float32')
    
    fc_layer_width = conv_embedded_params["fc_layer_width"]
    full_text_conv_layer_width = conv_embedded_params["full_text_conv_layer_width"]
    context_conv_layer_width = conv_embedded_params["context_conv_layer_width"]
    fc_layer_depth = conv_embedded_params["fc_layer_depth"]
    
    
    full_txt_embedding = Embedding(max_features,
                    embedding_dims,
                    weights=[embedding_matrix],
                    input_length=full_txt_maxlen,
                    name="full_txt_embedding",)
    context_txt_embedding = Embedding(max_features,
                    embedding_dims,
                    weights=[embedding_matrix],
                    input_length=context_txt_maxlen,
                    name="context_txt_maxlen",)
    
    full_txt_embedding_out = full_txt_embedding(full_txt_input)
    
    context_txt_embedding_out = context_txt_embedding(context_txt_input) # 25 channels
    
    full_txt_embedding_out = Dropout(0.25)(full_txt_embedding_out)
    context_txt_embedding_out = Dropout(0.25)(context_txt_embedding_out)
    
    # https://stackoverflow.com/questions/53849829/element-wise-multiplication-with-keras
    # print(K.int_shape(full_txt_gaussian),K.int_shape(full_txt_embedding_out))
    
    gaussian_mask_out = Lambda(lambda x: x[0] * K.expand_dims(x[1], axis=-1))([full_txt_embedding_out, full_txt_gaussian])
    full_txt_gaussian_out = Reshape((full_txt_maxlen,1))(full_txt_gaussian) 
    full_txt_mask_out = Reshape((full_txt_maxlen,1))(full_txt_mask) 
    
    
    fc_input1 = GlobalAveragePooling1D()(gaussian_mask_out)
    fc_input1 = fc_layer(fc_input1,fc_layer_width,)
    
    full_txt_embedding_masked = concatenate([full_txt_mask_out,full_txt_embedding_out,full_txt_gaussian_out]) # 27 channels now
    
    
    
    fc_input2 = conv_pipe(context_txt_embedding_out, context_conv_layer_width)
    fc_input3 = conv_pipe(full_txt_embedding_masked, full_text_conv_layer_width)
    
    fc_input1 = Dropout(0.2)(fc_input1)
    fc_input2 = Dropout(0.2)(fc_input2)
    fc_input3 = Dropout(0.2)(fc_input3)
    
    numerics = fc_layer(numeric_inputs,8,)
    numerics = fc_layer(numerics,8,)
    
    fc_input = concatenate([numerics,fc_input1,fc_input2,fc_input3])
    
    for i in range(fc_layer_depth-1):
        fc_input = fc_layer(fc_input,fc_layer_width)
    
    fc_out = fc_layer(fc_input,int(fc_layer_width/2),bn=False)
    fc_out = Dense(3)(fc_out)
    out = Activation("sigmoid")(fc_out) # vs sigmoid
    
    model = Model(inputs=[full_txt_input,full_txt_mask,full_txt_gaussian,context_txt_input,numeric_inputs], 
                  outputs=[out])
    
    return model

In [ ]:
gc.collect()
conv_embedded_params
looped_random_validation(df,build_model,train_generator_model_with_validation,extract_values,
                        **conv_embedded_params["training_policy"])

5486662

{'full_text_conv_layer_width': 8,
 'context_conv_layer_width': 6,
 'fc_layer_width': 80,
 'fc_layer_depth': 2,
 'training_policy': {'epochs': 20,
  'batch_size': 64,
  'policy': 'adam',
  'lr': 0.002,
  'max_lr_olr': 0.005,
  'lr_shed_fn': <function params.<lambda>(e, lr)>}}

Before Resampling, Size =  (3695, 18) 
 2    2692
1     585
0     418
Name: sentiment, dtype: int64
After Resampling =  (8543, 18) 
 0    2926
1    2925
2    2692
Name: sentiment, dtype: int64
Epoch 1/20
134/134 [==============================] - 106s 791ms/step - loss: 1.0824 - acc: 0.4142 - val_loss: 1.0284 - val_acc: 0.4686
Validation F1 Score = 0.3800, Train F1 Score = 0.4174
Epoch 2/20
134/134 [==============================] - 44s 327ms/step - loss: 0.9510 - acc: 0.5523 - val_loss: 0.9717 - val_acc: 0.5769
Validation F1 Score = 0.4920, Train F1 Score = 0.5834
Epoch 3/20
134/134 [==============================] - 42s 310ms/step - loss: 0.7762 - acc: 0.6713 - val_loss: 0.9404 - val_acc: 0.5870
Validation F1 Score = 0.5002, Train F1 Score = 0.6593
Epoch 4/20
134/134 [==============================] - 40s 296ms/step - loss: 0.6247 - acc: 0.7483 - val_loss: 1.0380 - val_acc: 0.6013
Validation F1 Score = 0.5039, Train F1 Score = 0.7176
Epoch 5/20
134/134 [==============================

```
Accuracy = 0.89, Macro F1 = 0.86
Accuracy = 0.65, Macro F1 = 0.52
              precision    recall  f1-score   support

           0       0.30      0.41      0.35       199
           1       0.41      0.51      0.45       248
           2       0.82      0.72      0.77      1137

   micro avg       0.65      0.65      0.65      1584
   macro avg       0.51      0.55      0.52      1584
weighted avg       0.69      0.65      0.66      1584
```

```
Accuracy = 0.92, Macro F1 = 0.90
Accuracy = 0.66, Macro F1 = 0.39
Accuracy = 0.67, Macro F1 = 0.51
```

In [20]:
model = build_model()
train_generator_model_with_validation(model,df,None,extract_values,
                   **conv_embedded_params["training_policy"])


Epoch 1/2
366/366 [==============================] - 41s 112ms/step - loss: 1.1686 - acc: 0.3845
 - lr: 0.01525 
Epoch 2/2
366/366 [==============================] - 32s 86ms/step - loss: 1.1251 - acc: 0.4472
 - lr: 0.00002 


In [ ]:

y_test_preds = model.predict(extract_values(df_test))
y_test_preds = np.argmax(y_test_preds, axis=1)
assert len(y_test_preds)==df_test.shape[0]
df_test['sentiment'] = y_test_preds
df_test[['unique_hash','sentiment']].to_csv("submission.csv",index=False)





# M1 variant: Only surrounding

In [134]:
def extract_values_only_surrounding(x):
    x4 = np.stack(x["context_txt_encoded"].values, axis=0)
    x5 = x[["occurences_count","review_length","review_length_by_occurences_count"]].values
    train_phase = False
    if 'ohe_labels' in x.columns:
        train_phase = True
        y = np.stack(x['ohe_labels'].values,axis=0)
    if train_phase:
        x4 = batch_cutout(x4, p=cutout_proba, min_words=min_cutout, max_words=max_cutout,)[0]
    
    if train_phase:
        return [x4,x5],y
    else:
        return [x4,x5]

In [146]:
def build_model_m1v():
    context_txt_input = Input(shape=(context_txt_maxlen,), dtype='int32')
    
    numeric_inputs = Input(shape=(3,),name="numeric_input", dtype='float32')
    
    fc_layer_width = conv_embedded_params_m1v1["fc_layer_width"]
    context_conv_layer_width = conv_embedded_params_m1v1["context_conv_layer_width"]
    fc_layer_depth = conv_embedded_params_m1v1["fc_layer_depth"]
    
    
    
    context_txt_embedding = Embedding(max_features,
                    embedding_dims,
                    weights=[embedding_matrix],
                    input_length=context_txt_maxlen,
                    name="context_txt_maxlen",)
    
    context_txt_embedding_out = context_txt_embedding(context_txt_input) # 25 channels
    context_txt_embedding_out = Dropout(0.2)(context_txt_embedding_out)
    
    # https://stackoverflow.com/questions/53849829/element-wise-multiplication-with-keras
    # print(K.int_shape(full_txt_gaussian),K.int_shape(full_txt_embedding_out))
    
    fc_input2 = conv_pipe(context_txt_embedding_out, context_conv_layer_width)
    fc_input2 = Dropout(0.2)(fc_input2)
    numerics = fc_layer(numeric_inputs,8,)
    numerics = fc_layer(numerics,8,)
    
    fc_input = concatenate([numerics,fc_input2])
    
    for i in range(fc_layer_depth-1):
        fc_input = fc_layer(fc_input,fc_layer_width)
    
    fc_out = fc_layer(fc_input,int(fc_layer_width/2),bn=False)
    fc_out = Dense(3)(fc_out)
    out = Activation("softmax")(fc_out)
    
    model = Model(inputs=[context_txt_input,numeric_inputs], 
                  outputs=[out])
    
    return model

In [147]:
gc.collect()
conv_embedded_params_m1v1
looped_random_validation(df,build_model_m1v,train_generator_model_with_validation,extract_values_only_surrounding,
                        **conv_embedded_params_m1v1["training_policy"])

2106

{'context_conv_layer_width': 8,
 'fc_layer_width': 80,
 'fc_layer_depth': 2,
 'training_policy': {'epochs': 20,
  'batch_size': 64,
  'policy': 'adam',
  'lr': 0.002,
  'max_lr_olr': 0.005,
  'lr_shed_fn': <function params.<lambda>(e, lr)>}}

Before Resampling, Size =  (3695, 18) 
 2    2659
1     605
0     431
Name: sentiment, dtype: int64
After Resampling =  (8701, 18) 
 1    3025
0    3017
2    2659
Name: sentiment, dtype: int64
Epoch 1/20
136/136 [==============================] - 47s 342ms/step - loss: 1.1243 - acc: 0.3822 - val_loss: 1.1457 - val_acc: 0.4658
Validation F1 Score = 0.3735, Train F1 Score = 0.3819
Epoch 2/20
136/136 [==============================] - 14s 106ms/step - loss: 1.0255 - acc: 0.4750 - val_loss: 1.1882 - val_acc: 0.4133
Validation F1 Score = 0.3789, Train F1 Score = 0.4346
Epoch 3/20
136/136 [==============================] - 14s 106ms/step - loss: 0.9207 - acc: 0.5615 - val_loss: 1.1083 - val_acc: 0.4438
Validation F1 Score = 0.4051, Train F1 Score = 0.4965
Epoch 4/20
136/136 [==============================] - 14s 105ms/step - loss: 0.7911 - acc: 0.6540 - val_loss: 1.0532 - val_acc: 0.4699
Validation F1 Score = 0.4304, Train F1 Score = 0.5729
Epoch 5/20
136/136 [==============================]

```
Accuracy = 0.91, Macro F1 = 0.88
Accuracy = 0.66, Macro F1 = 0.53
              precision    recall  f1-score   support

           0       0.30      0.42      0.35       189
           1       0.42      0.52      0.46       252
           2       0.82      0.73      0.77      1143

   micro avg       0.66      0.66      0.66      1584
   macro avg       0.51      0.55      0.53      1584
weighted avg       0.70      0.66      0.67      1584

```

```
Accuracy = 0.87, Macro F1 = 0.82
Accuracy = 0.67, Macro F1 = 0.49
Accuracy = 0.69, Macro F1 = 0.53
              precision    recall  f1-score   support

           0       0.36      0.35      0.36       113
           1       0.42      0.44      0.43       179
           2       0.80      0.80      0.80       764

   micro avg       0.69      0.69      0.69      1056
   macro avg       0.53      0.53      0.53      1056
weighted avg       0.69      0.69      0.69      1056
```

In [ ]:
gc.collect()
model = build_model_m1v()
train_generator_model_with_validation(model,df,None,extract_values_only_surrounding,
                   **conv_embedded_params_m1v1["training_policy"])


In [411]:

y_test_preds = model.predict(extract_values_only_surrounding(df_test))
y_test_preds = np.argmax(y_test_preds, axis=1)
assert len(y_test_preds)==df_test.shape[0]
df_test['sentiment'] = y_test_preds
df_test[['unique_hash','sentiment']].to_csv("submission.csv",index=False)



# M2: CNN Model - No Padding and Embedding layer

In [613]:
df_np = read_csv("train_non_padded.csv")
df_test_np = read_csv("test_non_padded.csv")

df_np.shape
df_test_np.shape

(5279, 14)

(2924, 12)

In [615]:
trained_embeddings = PreTrainedEmbeddingsTransformer(fb_model,size=fasttext_dims)

In [749]:


def extract_values_pretrained(x):
    x1 = x["full_txt"].values
    x2 = x["full_txt_mask"].values
    x3 = x["full_txt_mask_gaussian"].values
    x4 = x["context_txt"].values
    x5 = x[["occurences_count","review_length","review_length_by_occurences_count"]].values
    train_phase = False
    if 'ohe_labels' in x.columns:
        train_phase = True
        y = np.stack(x['ohe_labels'].values,axis=0)

    

    full_txt_maxlen = 0
    for x in x1:
        full_txt_maxlen = max(full_txt_maxlen,len(x))
        
    context_txt_maxlen = 0
    
    x1 = list(pad_text_sequences(x1, maxlen=full_txt_maxlen,jobs=1))
    x2 = list(pad_text_sequences(x2, maxlen=full_txt_maxlen,empty=0.0,jobs=1))
    x3 = list(pad_text_sequences(x3, maxlen=full_txt_maxlen,empty=0.0,jobs=1))
    x4 = list(pad_text_sequences(x4, maxlen=maxlen,jobs=1))
    
    x1 = np.array(x1)
    x4 = np.array(x4)
    
    if train_phase:
        x1,x2,x3 = batch_cutout( x1,x2,x3,p=cutout_proba, min_words=min_cutout, max_words=max_cutout,)
        
        x4 = batch_cutout(x4, p=cutout_proba, min_words=min_cutout, max_words=max_cutout,)[0]
    
    
    x4 = trained_embeddings.transform(x4)
    x1 = trained_embeddings.transform(x1)
    
    # Batch padding!!
    
    
    x1 = np.stack(x1, axis=0)
    x4 = np.stack(x4, axis=0)
    
    
    
    
    x2 = np.asarray([np.asarray(x).reshape((len(x),1)) for x in x2])
    x3 = np.asarray([np.asarray(x).reshape((len(x),1)) for x in x3])
    x2 = np.stack(x2, axis=0)
    x3 = np.stack(x3, axis=0)
    # print(x1.shape,x2.shape,x3.shape,x4.shape)
    if train_phase:
        return [x1,x2,x3,x4,x5],y
    else:
        return [x1,x2,x3,x4,x5]



In [750]:
sample = df_np.sample(1000)
_ = extract_values_pretrained(sample)

gc.collect()

12

In [751]:
def build_model_no_embedding():
    full_txt_input = Input(shape=(None,fasttext_dims), dtype='float32')
    full_txt_mask = Input(shape=(None,1), dtype='float32')
    full_txt_gaussian = Input(shape=(None,1), dtype='float32')
    
    context_txt_input = Input(shape=(None,fasttext_dims), dtype='float32')
    
    numeric_inputs = Input(shape=(3,),name="numeric_input", dtype='float32')
    
    fc_layer_width = conv_non_embedded_params["fc_layer_width"]
    full_text_conv_layer_width = conv_non_embedded_params["full_text_conv_layer_width"]
    context_conv_layer_width = conv_non_embedded_params["context_conv_layer_width"]
    fc_layer_depth = conv_non_embedded_params["fc_layer_depth"]
    
    
    full_txt_embedding_out = full_txt_input
    context_txt_embedding_out = context_txt_input # 25 channels
    
    full_txt_embedding_out = Dropout(0.1)(full_txt_embedding_out)
    context_txt_embedding_out = Dropout(0.1)(context_txt_embedding_out)
    print(K.int_shape(full_txt_embedding_out))
    
    # https://stackoverflow.com/questions/53849829/element-wise-multiplication-with-keras
    gaussian_mask_out = Lambda(lambda x: x[0] * x[1])([full_txt_embedding_out, full_txt_gaussian])
    print(K.int_shape(gaussian_mask_out))
    full_txt_gaussian_out = full_txt_gaussian
    full_txt_mask_out = full_txt_mask
    print(K.int_shape(full_txt_mask_out))
    
    fc_input1 = GlobalAveragePooling1D()(gaussian_mask_out)
    fc_input1 = fc_layer(fc_input1,fc_layer_width,)
    
    full_txt_embedding_masked = concatenate([full_txt_mask_out,full_txt_embedding_out,full_txt_gaussian_out]) # 27 channels now
    print(K.int_shape(full_txt_embedding_masked))
    
    fc_input2 = conv_pipe(context_txt_embedding_out, context_conv_layer_width)
    fc_input3 = conv_pipe(full_txt_embedding_masked, full_text_conv_layer_width)
    
    fc_input2 = Dropout(0.1)(fc_input2)
    numerics = fc_layer(numeric_inputs,8,)
    numerics = fc_layer(numerics,8,)
    
    fc_input = concatenate([numerics,fc_input1,fc_input2,fc_input3])
    
    for i in range(fc_layer_depth-1):
        fc_input = fc_layer(fc_input,fc_layer_width)
    
    fc_out = fc_layer(fc_input,int(fc_layer_width/2),bn=False)
    fc_out = Dense(3)(fc_out)
    out = Activation("softmax")(fc_out)
    
    model = Model(inputs=[full_txt_input,full_txt_mask,full_txt_gaussian,context_txt_input,numeric_inputs], 
                  outputs=[out])
    
    return model


In [ ]:
gc.collect()
print(conv_non_embedded_params)
looped_random_validation(df_np,build_model_no_embedding,train_generator_model_with_validation,extract_values_pretrained,
                        **conv_non_embedded_params["training_policy"])


0

{'full_text_conv_layer_width': 6, 'context_conv_layer_width': 4, 'fc_layer_width': 80, 'fc_layer_depth': 2, 'training_policy': {'epochs': 10, 'batch_size': 32, 'policy': 'adam', 'lr': 0.005, 'max_lr_olr': 0.005, 'lr_shed_fn': <function <lambda> at 0x7fae86602d90>}}
(None, None, 300)
(None, None, 300)
(None, None, 1)
(None, None, 302)
Before Resampling = 
 2    2744
1     619
0     437
Name: sentiment, dtype: int64
After Resampling = 
 1    3095
0    3059
2    2744
Name: sentiment, dtype: int64
Epoch 1/10
  8/279 [..............................] - ETA: 4:28:18 - loss: 1.3198 - acc: 0.3203

In [ ]:
model = build_model_no_embedding()
train_generator_model_with_validation(model,df_np,None,extract_values_pretrained,
                   **conv_non_embedded_params["training_policy"])


In [ ]:
iterator = MakeIter(df_test_np,extract_values,batch_size)
y_test_preds = model.predict_generator(iterator,steps=len(iterator))
y_test_preds = np.argmax(y_test_preds, axis=1)
df_test_np['sentiment'] = y_test_preds
df_test_np[['unique_hash','sentiment']].to_csv("submission.csv",index=False)


# M3: XGB Model

In [195]:
df_xgb = read_csv("train_non_padded.csv")
df_test_xgb = read_csv("test_non_padded.csv")


doc2vec = PreTrainedDocEmbeddingsTransformer(fb_model,start=0,end=300)

def append_doc2vec_columns(df,column):
    results = doc2vec.transform(df[column])
    text_df = pd.DataFrame(list(map(list, results)))
    text_df.columns = [column +"_doc2vec_"+ str(i) for i in range(0, doc2vec.size)]
    text_df.index = df.index
    df[list(text_df.columns)] = text_df
    return df
    

df_xgb = append_doc2vec_columns(df_xgb,column="full_txt")
df_xgb = append_doc2vec_columns(df_xgb,column="context_txt")
df_test_xgb = append_doc2vec_columns(df_test_xgb,column="full_txt")
df_test_xgb = append_doc2vec_columns(df_test_xgb,column="context_txt")

In [196]:
# xgb_features = df_utils.get_specific_cols(df_xgb,prefix="full_txt_doc2vec_") + df_utils.get_specific_cols(df_xgb,prefix="context_txt_doc2vec_") +["occurences_count","review_length","review_length_by_occurences_count"]
xgb_features = df_utils.get_specific_cols(df_xgb,prefix="context_txt_doc2vec_") +["occurences_count","review_length","review_length_by_occurences_count"]
target = "ohe_labels"
len(xgb_features)

303

In [197]:
def train_xgb(model,train):
    train = resample_input_for_training(train)
    y = np.asarray([x for x in train[target].values])
    train = train[xgb_features]
    model.fit(train,y)


In [198]:
def xgb_builder():
    # objective="binary:logitraw",
    model = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=6,scale_pos_weight=0.5,
                                              booster='dart',n_estimators=100,learning_rate=0.1,
                                              tree_method="exact",
                                             subsample=0.75,gamma=8,colsample_bylevel=0.5,colsample_bytree=0.5,
                                             alpha=0.4,num_parallel_tree=2))
    return model

In [199]:
def looped_xgb_validation(looping=5):
    model = xgb_builder()
    fscores = []
    for i in range(looping):
        np.random.seed(i*17 + 5*13+np.random.randint(10000))
        df_train, df_val = train_test_split(df_xgb,test_size=0.2)
        
        train_xgb(model,df_train)

        y_train_preds = model.predict_proba(df_train[xgb_features])
#         print(y_train_preds)
        y_train_preds = np.argmax(y_train_preds, axis=1)
        
        _ = show_results(df_train['sentiment'], y_train_preds)

        
        y_val_preds = model.predict_proba(df_val[xgb_features])
        y_val_preds = np.argmax(y_val_preds, axis=1)
        acc,f1_test = show_results(df_val['sentiment'], y_val_preds)
        model = xgb_builder()
        fscores.append(f1_test)
    print("Mean Fscores = ",np.mean(fscores))
        

In [200]:
looped_xgb_validation()

# max_depth = 6 0.47385244733641335
# max_depth = 4 0.4758742505101491

# 0.47720480008127925

Accuracy = 0.92, Macro F1 = 0.90
Accuracy = 0.63, Macro F1 = 0.46
Accuracy = 0.92, Macro F1 = 0.90
Accuracy = 0.63, Macro F1 = 0.48
Accuracy = 0.91, Macro F1 = 0.89
Accuracy = 0.64, Macro F1 = 0.47
Accuracy = 0.92, Macro F1 = 0.90
Accuracy = 0.63, Macro F1 = 0.48
Accuracy = 0.92, Macro F1 = 0.90
Accuracy = 0.65, Macro F1 = 0.50
Mean Fscores =  0.4761089456811837


**Methods**
- Use Keras Layer to generate vector representation of words
- Use FB Fasttext with GAP, GMP of both full_txt and context_txt
- No Padding Needed

# M5: BiDirectional LSTM Model

In [576]:
def build_lstm_model():
    full_txt_input = Input(shape=(full_txt_maxlen,), dtype='int32')
    full_txt_mask = Input(shape=(full_txt_maxlen,), dtype='float32')
    full_txt_gaussian = Input(shape=(full_txt_maxlen,), dtype='float32')
    context_txt_input = Input(shape=(context_txt_maxlen,), dtype='int32')
    
    numeric_inputs = Input(shape=(3,),name="numeric_input", dtype='float32')
    
    fc_layer_width = lstm_embedded_params["fc_layer_width"]
    lstm_context_units = lstm_embedded_params["lstm_context_units"]
    lstm_full_text_units = lstm_embedded_params["lstm_full_text_units"]
    fc_layer_depth = lstm_embedded_params["fc_layer_depth"]
    
    
    full_txt_embedding = Embedding(max_features,
                    embedding_dims,
                    weights=[embedding_matrix],
                    input_length=full_txt_maxlen,
                    name="full_txt_embedding",)
    context_txt_embedding = Embedding(max_features,
                    embedding_dims,
                    weights=[embedding_matrix],
                    input_length=context_txt_maxlen,
                    name="context_txt_maxlen",)
    
    full_txt_embedding_out = full_txt_embedding(full_txt_input)
    context_txt_embedding_out = context_txt_embedding(context_txt_input) # 25 channels
    context_txt_embedding_out = Dropout(0.3)(context_txt_embedding_out)
    full_txt_embedding_out = Dropout(0.3)(full_txt_embedding_out)
    
    
    
    # https://stackoverflow.com/questions/53849829/element-wise-multiplication-with-keras
    # print(K.int_shape(full_txt_gaussian),K.int_shape(full_txt_embedding_out))
    
    gaussian_mask_out = Lambda(lambda x: x[0] * K.expand_dims(x[1], axis=-1))([full_txt_embedding_out, full_txt_gaussian])
    full_txt_gaussian_out = Reshape((full_txt_maxlen,1))(full_txt_gaussian) 
    full_txt_mask_out = Reshape((full_txt_maxlen,1))(full_txt_mask) 
    
    
    fc_input1 = GlobalAveragePooling1D()(gaussian_mask_out)
    fc_input1 = fc_layer(fc_input1,fc_layer_width,)
    
    full_txt_embedding_masked = concatenate([full_txt_mask_out,full_txt_embedding_out,full_txt_gaussian_out]) # 27 channels now
    # print(K.int_shape(full_txt_embedding_masked))
    fc_input2 = Bidirectional(LSTM(lstm_context_units,dropout=0.1, return_sequences=True))(context_txt_embedding_out)
    fc_input3 = Bidirectional(LSTM(lstm_full_text_units, dropout=0.1, return_sequences=True))(full_txt_embedding_masked)
    
    fc_input2 = Bidirectional(LSTM(lstm_context_units,dropout=0.1))(fc_input2)
    fc_input3 = Bidirectional(LSTM(lstm_full_text_units, dropout=0.1))(fc_input3)
    
    
    
    fc_input1 = Dropout(0.1)(fc_input1)
    fc_input2 = Dropout(0.1)(fc_input2)
    numerics = fc_layer(numeric_inputs,8,)
    numerics = fc_layer(numerics,8,)
    
    fc_input = concatenate([numerics,fc_input1,fc_input2,fc_input3])
    
    for i in range(fc_layer_depth-1):
        fc_input = fc_layer(fc_input,fc_layer_width)
    
    fc_out = fc_layer(fc_input,int(fc_layer_width/2),bn=False)
    fc_out = Dense(3)(fc_out)
    out = Activation("softmax")(fc_out)
    
    model = Model(inputs=[full_txt_input,full_txt_mask,full_txt_gaussian,context_txt_input,numeric_inputs], 
                  outputs=[out])
    
    
    return model


In [ ]:
gc.collect()
print(lstm_embedded_params)
looped_random_validation(build_lstm_model,train_generator_model_with_validation,extract_values,
                        **lstm_embedded_params["training_policy"])


11460

{'lstm_full_text_units': 64, 'lstm_context_units': 32, 'fc_layer_width': 32, 'fc_layer_depth': 3, 'training_policy': {'epochs': 20, 'batch_size': 64, 'policy': 'adam', 'lr': 0.001, 'max_lr_olr': 0.005, 'lr_shed_fn': <function <lambda> at 0x7faf35185488>}}
Before Resampling = 
 2    2754
1     606
0     440
Name: sentiment, dtype: int64
After Resampling = 
 2    2754
1    2424
0    2200
Name: sentiment, dtype: int64
Epoch 1/20
116/116 [==============================] - 750s 6s/step - loss: 1.1535 - acc: 0.3411 - val_loss: 1.1128 - val_acc: 0.3036
Validation F1 Score = 0.2434, Train F1 Score = 0.2547
Epoch 2/20
116/116 [==============================] - 444s 4s/step - loss: 1.1059 - acc: 0.3810 - val_loss: 1.0714 - val_acc: 0.4460
Validation F1 Score = 0.2895, Train F1 Score = 0.3072
Epoch 3/20
116/116 [==============================] - 443s 4s/step - loss: 1.0878 - acc: 0.3805 - val_loss: 1.1055 - val_acc: 0.3696
Validation F1 Score = 0.2598, Train F1 Score = 0.3303
Epoch 4/20
116/116 [

# M6: LSTM No Embedding Model

In [ ]:
def build_lstm_no_embedding():
    full_txt_input = Input(shape=(None,fasttext_dims), dtype='float32')
    full_txt_mask = Input(shape=(None,1), dtype='float32')
    full_txt_gaussian = Input(shape=(None,1), dtype='float32')
    
    
    context_txt_input = Input(shape=(None,fasttext_dims), dtype='float32')
    

    numeric_inputs = Input(shape=(3,),name="numeric_input", dtype='float32')

    fc_layer_width = lstm_non_embedded_params["fc_layer_width"]
    lstm_context_units = lstm_non_embedded_params["lstm_context_units"]
    lstm_full_text_units = lstm_non_embedded_params["lstm_full_text_units"]
    fc_layer_depth = lstm_non_embedded_params["fc_layer_depth"]
    
    
    full_txt_embedding_out = full_txt_input
    context_txt_embedding_out = context_txt_input # 25 channels
    
    
    # https://stackoverflow.com/questions/53849829/element-wise-multiplication-with-keras
    # print(K.int_shape(full_txt_gaussian),K.int_shape(full_txt_embedding_out))
    # gaussian_mask_out = Lambda(lambda x: x[0] * K.expand_dims(x[1], axis=-1))([full_txt_embedding_out, full_txt_gaussian])
    gaussian_mask_out = Lambda(lambda x: x[0] * x[1])([full_txt_embedding_out, full_txt_gaussian])
    print(K.int_shape(gaussian_mask_out))
    full_txt_gaussian_out = full_txt_gaussian
    full_txt_mask_out = full_txt_mask
    # full_txt_gaussian_out = Reshape((-1,1))(full_txt_gaussian) 
    # full_txt_mask_out = Reshape((-1,1))(full_txt_mask) 
    
    print(K.int_shape(gaussian_mask_out))
    fc_input1 = GlobalAveragePooling1D()(gaussian_mask_out)
    fc_input1 = fc_layer(fc_input1,fc_layer_width,)
    
    full_txt_embedding_masked = concatenate([full_txt_mask_out,full_txt_embedding_out,full_txt_gaussian_out]) # 27 channels now
    print(K.int_shape(full_txt_embedding_out),K.int_shape(full_txt_embedding_masked))
    
    fc_input2 = Bidirectional(LSTM(lstm_context_units))(context_txt_embedding_out)
    fc_input3 = Bidirectional(LSTM(lstm_full_text_units))(full_txt_embedding_masked)
    
    fc_input1 = Dropout(0.1)(fc_input1)
    fc_input2 = Dropout(0.1)(fc_input2)
    fc_input3 = Dropout(0.1)(fc_input3)
    numerics = fc_layer(numeric_inputs,8,)
    numerics = fc_layer(numerics,8,)
    
    
    fc_input = concatenate([numerics,fc_input1,fc_input2,fc_input3])
    
    for i in range(fc_layer_depth-1):
        fc_input = fc_layer(fc_input,fc_layer_width)
    
    fc_out = fc_layer(fc_input,int(fc_layer_width/2),bn=False)
    fc_out = Dense(3)(fc_out)
    out = Activation("softmax")(fc_out)
    
    model = Model(inputs=[full_txt_input,full_txt_mask,full_txt_gaussian,context_txt_input,numeric_inputs], 
                  outputs=[out])
    
    
    
    
    return model

In [ ]:
gc.collect()
print(lstm_non_embedded_params)
looped_random_validation(df_np,build_lstm_no_embedding,train_generator_model_with_validation,extract_values_pretrained,
                        **lstm_non_embedded_params["training_policy"])


# Hierarchical Tree Model

In [549]:
df_xgb = read_csv("train_non_padded.csv")
df_test_xgb = read_csv("test_non_padded.csv")


doc2vec = PreTrainedDocEmbeddingsTransformer(fb_model,start=0,end=300)

def append_doc2vec_columns(df,column):
    results = doc2vec.transform(df[column])
    text_df = pd.DataFrame(list(map(list, results)))
    text_df.columns = [column +"_doc2vec_"+ str(i) for i in range(0, doc2vec.size)]
    text_df.index = df.index
    df[list(text_df.columns)] = text_df
    return df
        

df_xgb = append_doc2vec_columns(df_xgb,column="full_txt")
df_xgb = append_doc2vec_columns(df_xgb,column="context_txt")
df_test_xgb = append_doc2vec_columns(df_test_xgb,column="full_txt")
df_test_xgb = append_doc2vec_columns(df_test_xgb,column="context_txt")

In [550]:
xgb_features = df_utils.get_specific_cols(df_xgb,prefix="context_txt_doc2vec_") +["occurences_count","review_length","review_length_by_occurences_count"]
target = "ohe_labels"
len(xgb_features)

303

In [559]:
def train_xgb(model_1,model_2,train):
    train = resample_input_for_training(train)
    y_s = np.asarray([x[2]==1 for x in train[target].values])
    y_n = np.asarray([x[1]==1 for x in train[target].values])
    
    y_i = np.asarray([x[1]==1 or x[0]==1 for x in train[target].values])
    train = train[xgb_features]
    model_1.fit(train,y_s.astype(int))
    model_2.fit(train[y_i],y_n[y_i])
    
def predict_xgb(model_1,model_2,df):
    y_s = model_1.predict(df)
    y_s[y_s==1] = 2
    
    y_2 = model_2.predict(df)
    y_s[y_s!=2] = y_2[y_s!=2]
    return y_s


In [566]:
def xgb_builder(scale_pos_weight):
    # objective="binary:logitraw",
    model = XGBClassifier(n_jobs=-1, max_depth=8, scale_pos_weight=scale_pos_weight,
                                              booster='dart',n_estimators=100,learning_rate=0.1,
                                              tree_method="exact",
                                             subsample=0.75,gamma=8,colsample_bylevel=0.5,colsample_bytree=0.5,
                                             alpha=0.4,num_parallel_tree=2)
    return model

In [573]:
def looped_xgb_validation(looping=5):
    
    fscores = []
    for i in range(looping):
        m1 = xgb_builder(1.7)
        m2 = xgb_builder(0.7)
        np.random.seed(i*17 + 5*13+np.random.randint(10000))
        df_train, df_val = train_test_split(df_xgb,test_size=0.2)
        
        train_xgb(m1,m2,df_train)

        y_train_preds = predict_xgb(m1,m2,df_train[xgb_features])
#         print(y_train_preds)
        # y_train_preds = np.argmax(y_train_preds, axis=1)
        
        _ = show_results(df_train['sentiment'], y_train_preds)

        
        y_val_preds = predict_xgb(m1,m2,df_val[xgb_features])
        # y_val_preds = np.argmax(y_val_preds, axis=1)
        acc,f1_test = show_results(df_val['sentiment'], y_val_preds)
        fscores.append(f1_test)
        print(classification_report(df_val['sentiment'], y_val_preds))
    print("Mean Fscores = ",np.mean(fscores))
    

In [574]:
looped_xgb_validation(5)

Accuracy = 0.99, Macro F1 = 0.99
Accuracy = 0.68, Macro F1 = 0.47
              precision    recall  f1-score   support

           0       0.25      0.23      0.24       120
           1       0.37      0.34      0.35       165
           2       0.80      0.82      0.81       771

   micro avg       0.68      0.68      0.68      1056
   macro avg       0.47      0.46      0.47      1056
weighted avg       0.67      0.68      0.67      1056

Accuracy = 0.99, Macro F1 = 0.99
Accuracy = 0.66, Macro F1 = 0.45
              precision    recall  f1-score   support

           0       0.27      0.17      0.21       143
           1       0.33      0.35      0.34       156
           2       0.77      0.82      0.79       757

   micro avg       0.66      0.66      0.66      1056
   macro avg       0.46      0.44      0.45      1056
weighted avg       0.64      0.66      0.65      1056

Accuracy = 0.99, Macro F1 = 0.99
Accuracy = 0.69, Macro F1 = 0.47
              precision    recall  f1-sc

In [531]:
3345 + 2856

6201